In [3]:
from google.cloud import bigquery
client = bigquery.Client()

query = ''' select DISTINCT ST_DOMAIN FROM TELCO_MDI.SMALL_MID_MARKET_RANDOM_SAMPLE where ST_DOMAIN IS NOT NULL LIMIT 100000'''
query_job = client.query(query,location = 'US')
df = query_job.to_dataframe()

In [3]:
df

ST_DOMAIN
0                   seeboldlaw.com
1          dlpratcherlawoffice.com
2                     qdro-law.com
3                      cycnlaw.com
4              wheelocklawfirm.com
...                            ...
9995              jjlovebeauty.com
9996         rolfinggreensboro.com
9997             bellableuespa.com
9998  incrediblemoldingfitness.net
9999          skinholistic-spa.com

[10000 rows x 1 columns]

In [ ]:

keyword_lst= ['Keywords',
'Appointment',
'Attention',
'Challenging Time',
'Closing',
'Company Update',
'Corona',
'Coronavirus',
'Covid',
'Critical',
'Curbside ',
'Delivery',
'Discount',
'Health',
'In-person',
'Layoff',
'Now Offering',
'Offering',
'Open',
'Outbreak',
'Pandemic',
'Safety',
'SARS-CoV-2',
'Stimulus',
'Update',
'Virus',
'Wellbeing']





In [2]:
#!/usr/bin/env python
# coding: utf-8

# ### Key word Crawler 
# 
# <b>Steps to follow :</b><br>
# 1. Upload input xlsx file having list of keywords, threads, input URL<br>
# 2. Please update the path of chromedriver, search for "chromedriver_path" in the code replace the existing path<br>
# 3. Please update the output path, where the results file has be stored<br>
# 4. After making the above changes Intialize/Run every cell in the sequence
# 

keyword_lst= ['Keywords',
'Appointment',
'Attention',
'Challenging Time',
'Closing',
'Company Update',
'Corona',
'Coronavirus',
'Covid',
'Critical',
'Curbside ',
'Delivery',
'Discount',
'Health',
'In-person',
'Layoff',
'Now Offering',
'Offering',
'Open',
'Outbreak',
'Pandemic',
'Safety',
'SARS-CoV-2',
'Stimulus',
'Update',
'Virus',
'Wellbeing']
# #### Importing all key packages

# In[113]:


import pandas as pd
import platform
import random
import re
import csv
import time
import PyPDF2
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from fuzzywuzzy import fuzz
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from threading import Thread
import concurrent.futures as ft
import requests
import time
import ast
import nltk
import itertools

nltk.download('punkt')


# #### Code body 

# In[162]:



def url_check(input_url):
    url_low = input_url.lower()
    #url_low = url_low.split(';')[0]
    if not url_low.startswith('http'):
        url_low = 'http://' + url_low
    return url_low

def get_pdftext(pdf_url):

    u = requests.get(pdf_url)
    f = BytesIO()
    f.write(u.content)
    pdfFile = PyPDF2.PdfFileReader(f)

    pdf_text = []
    for pageNum in range(pdfFile.getNumPages()):
        currentPage = pdfFile.getPage(pageNum)
        #currentPage.mergePage(watermark.getPage(0))
        pdf_text.append(currentPage.extractText())

    return '\n'.join(pdf_text)

## driversList would be a global var
## driverLock using lock capability of multithreading package ## checking in documentation

def getDriver(i,lock):
    global driversList
    global driverCnt
    driverReady = False
    driverMax = 5
    while not driverReady:
        driverReady = True
        with lock:
            if len(driversList):
                tempD = driversList.pop()
            elif driverCnt < driverMax:
                print("creating new")
                #init driver
                chrome_options = webdriver.chrome.options.Options()
                chrome_options.add_argument('--dns-prefetch-disable')
                chrome_options.add_argument('--no-sandbox')
                chrome_options.add_argument('--lang=en-US')
                #chrome_options.add_argument('--headless')
                chrome_options.add_argument('--disable-gpu')

                tempD = webdriver.Chrome(executable_path='chromedriver.exe', options=chrome_options)
                driverCnt += 1
            else:
                driverReady = False
    return(tempD)



class CovidCrawler():

    display_name = "keyword Crawler"
    tag_name = "keyword_crawler"
    requires = []
    config = {}


    def key_word_match(self, text):
        # text = text.replace('\n',' ')
        text_len = len(text)
        text_lower = text.lower()

        is_output = False
        # output_dict = {}
        sent_text = nltk.sent_tokenize(text.replace("\n","").replace("\r","").replace("\t"," ")) 
        
        output_list = []
        cnt_sent = 0 
        for sent in sent_text:
            if any(x in sent.lower() for x in self.search_words):
                output_list.append(sent.replace("   ",""))
                cnt_sent = cnt_sent + 1
                
        
        
        for each_keyword in self.search_words :
            search = re.finditer(each_keyword.lower(),text_lower)
            cnt = 0
            matching = []
            for m in search:
                is_output = True
                cnt = cnt + 1

                start_idx = m.start()
                end_idx = m.end()

                start_idx = start_idx - self.win_size
                end_idx = end_idx + self.win_size

                if start_idx < 0:
                    start_idx = 0
                if end_idx > text_len-1:
                    end_idx = text_len-1

                matching.append(text[start_idx:end_idx].replace("\n","").replace("\r","").replace("   ",""))
            output_list.append(matching)
            
        if len(output_list) >=1:
            output_list = list(filter(None, output_list)) 
            cnt = len(output_list)

        return output_list, is_output, cnt

    def href_check_w_keywords(self):

        urls = []
        cnt_url= 0
        html_a = self.soup.findAll('a', href = True)
        for each_a in html_a:
            text = each_a.text.lower()
            href = each_a['href']
            if not href:
                continue

            for each_keyword in self.url_search_words:
                if each_keyword.lower() in text or each_keyword.lower() in href.lower():
                    urls.append(href)
                    cnt_url = cnt_url +1

        urls = list(set(urls))

        return urls,cnt_url

    def extract_info(self,i,url,lock):
        
        if str(url) !='nan':
            old_url= url
            url= url_check(url)

            if (i %1000 == 0):
                print(i,url)


            user_agent='Chrome/35.0.1916.47 Chrome/57.0.2987.110 Chrome/61.0.3163.91 Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/537.36'
            headers = {'User-Agent': user_agent,'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}

            self.search_words = main_search_words
            self.url_search_words = url_search_words
            self.win_size = main_page_char_win

            is_crawled = '1'
            #main page search
            try:
                req = requests.get(url.lower(),headers=headers,timeout=30)
                
                self.soup = BeautifulSoup(req.content, 'lxml')
            #time.sleep(1)
                self.body = self.soup.find('body')
                raw_text = self.body.get_text()
            except:
                try:
#                     chrome_options = webdriver.chrome.options.Options()
#                     chrome_options.add_argument('--dns-prefetch-disable')
#                     chrome_options.add_argument('--no-sandbox')
#                     chrome_options.add_argument('--lang=en-US')
#                     chrome_options.add_argument('--headless')
#                     chrome_options.add_argument('--disable-gpu')
#                     self.driver = webdriver.Chrome(executable_path=chromedriver_path, options=chrome_options)
                    self.driver = getDriver()
                    self.driver.get(url.lower())
                    self.soup = BeautifulSoup(self.driver.page_source, 'lxml')
                    driversList.append(self.driver)
                    self.body = self.soup.find('body')
                    raw_text = self.body.get_text()
                    self.driver.close()
                except:
                    raw_text = ""
                    self.body = ""

            if (raw_text=='') or (str(req) != '<Response [200]>') or any(x in raw_text.lower() for x in ["blocked","bad url","(error 404)","403 error","doesn't exist (404)","404 not found","page not found","access denied","database error","this page isn't available"]):
                is_crawled = '0'
                output_text = [""]
                url_list = [""]
                cnt_sent = ""
                cnt_url = ""


            else :
                try:
                    output_text = []
                    output_list, is_output, cnt_sent = self.key_word_match(self.body.get_text())
                    output_text.append(output_list)
                except:
                    output_text = [""]


                # sub page search
                try:
                    url_list = []
                    covid_urls, cnt_url = self.href_check_w_keywords()
                    url_list.append(covid_urls)
                except:
                    url_list = [""]
           

            temp.at[i,'output_text'] = output_list
            temp.at[i,'sub_url_list'] = covid_urls
            temp.at[i,'Resolve'] = is_crawled
            temp.at[i,'# of sentences having keyword'] =cnt_sent
            temp.at[i,'# of URLs having keyword']= cnt_url
            
        else:
            is_crawled = '0'
            output_text = [""]
            output_list = ""
            covid_urls = ""
            url_list = [""]
            cnt_sent = ""
            cnt_url = ""
            
            temp.at[i,'output_text'] = output_list
            temp.at[i,'sub_url_list'] = covid_urls
            temp.at[i,'Resolve'] = is_crawled
            temp.at[i,'# of sentences having keyword'] =cnt_sent
            temp.at[i,'# of URLs having keyword']= cnt_url

# In[167]:


if __name__ == "__main__":
    # execute only if run as a script
    
    ### update input and output file paths 
    #input_excel_file = 'PLEASE PLACE INPUT FILE PATH'
    output_csv_file = 'D:/Telco MD&I/Covid crawler/output_VZ_keyword_1M.csv'
    chromedriver_path = 'D:/Telco MD&I/Code/chromedriver.exe'
    df = df
 
    
    #### Parameter initialziations
#     main_search_words = input_excel_file['Keywords']
#     url_search_words = input_excel_file['Keywords']
#     threads = input_excel_file['Threads']
    main_search_words = keyword_lst
    url_search_words = keyword_lst
    threads = 100
    main_page_char_win = 200
    google_search = False
    covid_crawler = CovidCrawler()
    start = time.perf_counter()
    temp = df[['ST_DOMAIN']] 
    concurrent = 500
    
    temp['output_text'] = 0
    temp['sub_url_list'] = 0
    temp['Resolve'] = 0
    temp['# of sentences having keyword'] =0
    temp['# of URLs having keyword']= 0
    
    lock =threading.Lock()
    driverCnt = 0
    driversList = []
    ##### MultiThreading code
    with ft.ThreadPoolExecutor(max_workers = threads) as executor:
        rt = executor.map(covid_crawler.extract_info,temp.index, temp['ST_DOMAIN'],itertools.repeat(lock))
    end = time.perf_counter()
    print(end-start)
#     lol = [ast.literal_eval(r) for r in rt]
#     df = [pd.DataFrame(s) for s in lol]
#     res = pd.concat(df)
#     res.columns  = ['url', 'output_text', 'sub_url_list', 'Resolve',
#        '# of sentences having keyword', '# of URLs having keyword']
#     res = res.reset_index(drop=True)
    temp.to_csv(output_csv_file)


# 1.	Update input_excel_file = ‘'D:/Telco MD&I/websites/Websites_15670.xlsx’
# 2.	Update output_excel_file = ‘'D:/Telco MD&I/Covid crawler/output.csv'’
# 3.	Keywords = [‘products’,’services’,’wireline’]
# 4.	Threads = 300 
# 5.	Update chromedriver_path = ‘D:/Telco MD&I/chromedriver.exe’


c:\users\v.behl\appdata\local\programs\python\python37\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\v.behl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


NameError: name 'df' is not defined

In [5]:
import threading
import pandas as pd
import platform
import random
import re
import csv
import time
import PyPDF2
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from fuzzywuzzy import fuzz
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from threading import Thread
import concurrent.futures as ft
import requests
import time
import ast
import nltk
import itertools

In [6]:
def getDriver(i,lock):
    global driversList
    global driverCnt
    driverReady = False
    driverMax = 5
    while not driverReady:
        driverReady = True
        with lock:
            if len(driversList):
                tempD = driversList.pop()
            elif driverCnt < driverMax:
                print("creating new")
                #init driver
                chrome_options = webdriver.chrome.options.Options()
                chrome_options.add_argument('--dns-prefetch-disable')
                chrome_options.add_argument('--no-sandbox')
                chrome_options.add_argument('--lang=en-US')
                #chrome_options.add_argument('--headless')
                chrome_options.add_argument('--disable-gpu')

                tempD = webdriver.Chrome(executable_path='chromedriver.exe', options=chrome_options)
                driverCnt += 1
            else:
                driverReady = False
    return(tempD)

def crawler(i,url,lock):
    
    global driversList
    url_low = url.lower()
    #url_low = url_low.split(';')[0]
    if not url_low.startswith('http'):
        url_low = 'http://' + url_low

    user_agent='Chrome/35.0.1916.47 Chrome/57.0.2987.110 Chrome/61.0.3163.91 Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/537.36'
    headers = {'User-Agent': user_agent,'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}

    is_crawled = '1'

#     except:
    try:
#                     chrome_options = webdriver.chrome.options.Options()
#                     chrome_options.add_argument('--dns-prefetch-disable')
#                     chrome_options.add_argument('--no-sandbox')
#                     chrome_options.add_argument('--lang=en-US')
#                     chrome_options.add_argument('--headless')
#                     chrome_options.add_argument('--disable-gpu')
#                     self.driver = webdriver.Chrome(executable_path=chromedriver_path, options=chrome_options)
        start = time.perf_counter()
        driver = getDriver(i,lock)
        print("driver start length:",len(driversList))
        print("using for 3 seconds :",url)
        time.sleep(3)
        #driver.get(url_low.lower())
        #soup = BeautifulSoup(driver.page_source, 'lxml')
        driversList.append(driver)
        end = time.perf_counter()
        print( "{} driver end : {} {}".format(url,driver,round(start- end),2),)
        #body = soup.find('body')
        #raw_text = body.get_text()
        #print(url,":",len(raw_text))
        driver.close()
    except:
        raw_text = ""
        body = ""
    return(len(body))

    
    

In [7]:
df = pd.read_csv("Dummy_data.csv")

In [8]:
df= df[0:50]

In [10]:
df

org_name  \
0                  BUILDING CONTRACTORS INC   
1                          ACTION LIFT  INC   
2                         HARDY DIAGNOSTICS   
3             HEALTH CARE INSURANCE COMPANY   
4                      METIS SOLUTIONS  LLC   
5            VNA HOME CARE OF MERCER COUNTY   
6                            ALLPRO IMAGING   
7           MAPLE PRESS DISTRIBUTION CENTER   
8                        NIAGARA SHEETS LLC   
9                  VIRGINIA PHYSICIANS  INC   
10                EXECUTIVE MAINTENANCE INC   
11              WHOLESOME INTERNATIONAL LLC   
12    BENS KOSHER DELI RESTAURANT  CATERERS   
13  SHOPRITE WINES  SPIRITS OF LITTLE FALLS   
14                     ATLANTIC RECORDS INC   
15                    NETWORK SOLUTIONS LLC   
16      MARY IMMACULATE HEALTHCARE SERVICES   
17                 SHOP N SAVE SUPERMARKETS   
18                       DREAM TRUCKING LLC   
19                     ADVENTIST HEALTHCARE   
20                              ARM  HAMMER   
21                 CHILDRENS MEDICAL CENTER   
22                                     CPSC   
23                     EVERYTHING BUT WATER   
24                        FALCON PROPERTIES   
25             HARTE HANKS DIRECT MARKETING   
26                 HIGH POINT SOLUTIONS LLC   
27                                NORDSTROM   
28                                      PBS   
29             PHILADELPHIA HOSPITALITY INC   
30                    ST CATHERINE OF SIENA   
31                 STERLING INFOSYSTEMS INC   
32                            TIO FOODS LLC   
33                          TRUSTPILOT  INC   
34                               VIVOX  INC   
35                BINGO TIRES  AUTO SERVICE   
36                 SILKROAD TECHNOLOGY  INC   
37              PYRAMID HEALTHCARE PRPTS LP   
38             JUDY DIAMOND ASSOCIATES  INC   
39                 DUCON ENVMTL SYSTEMS INC   
40                           NYACK HOSPITAL   
41                            PAUL L AXTELL   
42                   BEAVERVISITEC INTL INC   
43              ALLIED INTEGRATED MARKETING   
44                  NEISHLOSS  FLEMING  LLC   
45                  SCHNEIDER DOWNS  CO INC   
46                     YMCA OF PHOENIXVILLE   
47                   INDEPENDENT COACH CORP   
48              GRAND BANKCARNEGIE MORTGAGE   
49                              GAMES X INC   

                                              website  
0                           rhbuildingcontractors.com  
1                        http://www.actionliftinc.com  
2                     http://www.hardydiagnostics.com  
3             https://www.religarehealthinsurance.com  
4                        http://www.metisolutions.com  
5                        https://www.vnaalliance.org/  
6                                  allpro-imaging.com  
7                       https://www.maplepress.co.in/  
8                        http://www.niagarasheets.com  
9                         http://www.vaphysicians.com  
10                        executivemaintenanceinc.net  
11                                        panjiva.com  
12                           http://www.bensdeli.net/  
13                        http://www.shopritewine.com  
14                         http://atlanticrecords.com  
15                           www.networksolutions.com  
16                             https://www.mihcs.org/  
17                          https://www.shopnsave.com  
18                          www.dreamcitytrucking.com  
19                 http://www.adventisthealthcare.com  
20                               www.armandhammer.com  
21                        www.childrensmedicalctr.com  
22                               http://www.cpsc.gov/  
23                         www.everythingbutwater.com  
24                               falconproperties.com  
25                          http://www.hartehanks.com  
26                  http://www.highpointsolutions.com  
27                          http://shop.nordstrom.com  
28           

In [11]:
lock =threading.Lock()
driverCnt = 0
driversList = []
with ft.ThreadPoolExecutor(max_workers = 10) as executor:
    executor.map(crawler,df.index, df.website,itertools.repeat(lock))
    #end = time.perf_counter()


creating new
driverCnt 1
driver start length: 0
using for 3 seconds : rhbuildingcontractors.com
creating new
driverCnt 2
driver start length: 0
using for 3 seconds : http://www.actionliftinc.com
creating new
driverCnt 3
driver start length: 0
using for 3 seconds : http://www.hardydiagnostics.com
creating new
driverCnt 4
driver start length: 0
using for 3 seconds : https://www.religarehealthinsurance.com
creating new
driverCnt 5
driver start length: 0
using for 3 seconds : http://www.metisolutions.com
rhbuildingcontractors.com driver end : driver0 -3seletcing driver from list :
 0
driver0
http://www.actionliftinc.com driver end : driver1 -3driver start length: 1seletcing driver from list : 0
driver1
driver start length: 0
using for 3 seconds : https://www.vnaalliance.org/


using for 3 seconds : https://www.maplepress.co.in/
http://www.hardydiagnostics.com driver end : driver2 -3seletcing driver from list : 0

driver2
driver start length: 0
using for 3 seconds : http://www.niagarasheets

In [ ]:
itertools.repeat(lock)

In [1]:
df

NameError: name 'df' is not defined

In [12]:
import itertools

In [12]:
base = pd.read_csv('D:\Telco MD&I\Look_alike_model\Output\VZ_leads_MM_MB_COMBINED_3P.csv')

In [16]:
base = base[base.segment == 'Mid-Market']

In [20]:
base.columns

Index(['COMPANY', 'STREET', 'CITY', 'STATE', 'VZ_Product', 'PB_MERGE_FLAG',
       'PB_HTTP', 'PB_ID', 'ST_DOMAIN', 'ST_MERGE_FLAG', 'ST_DUNSNUMBER',
       'v12_web_address', 'V12_MERGE_FLAG', 'v12_company_id', 'Final_url',
       'segment', 'merge'],
      dtype='object')

In [23]:
base[base.Final_url.isna()==False]

COMPANY                STREET            CITY STATE  \
20866       PARADIGM FUNDS             9 ELK ST           ALBANY    NY   
20869             THE BANK         100 PARK AVE         WOODBURY    NJ   
20870         BB GROUP LLC  8002 KEW GARDENS RD      KEW GARDENS    NY   
20872          N B LIEBMAN   4705 CARLISLE PIKE    MECHANICSBURG    PA   
20873         CERMELE CAFE       303 S BROADWAY        TARRYTOWN    NY   
...                    ...                   ...             ...   ...   
27413  WARREN DISTRIBUTING       200 S ROUTE 73        HAMMONTON    NJ   
27414  WEIS MARKETS BAKERY    5000 JONESTOWN RD       HARRISBURG    PA   
27423           DDM ENERGY      841 LINCOLN AVE     WEST CHESTER    PA   
27425   FLORAL REFLECTIONS        125 W BEAU ST       WASHINGTON    PA   
27427             INFINITI   1501 COTTONTAIL LN         SOMERSET    NJ   

                   VZ_Product  PB_MERGE_FLAG                  PB_HTTP  \
20866  Internet New Net Promo            1.0  WWW.PARADIGMCAPITAL.COM   
20869  Internet New Net Promo            1.0                      NaN   
20870  Internet New Net Promo            1.0                      NaN   
20872  Internet New Net Promo            NaN                      NaN   
20873  Internet New Net Promo            1.0     CERMELESCATERING.COM   
...                       ...            ...                      ...   
27413  Internet New Net Promo            1.0   WWW.WARRENBEER.COM/WS/   
27414  Internet New Net Promo            1.0      WWW.WEISMARKETS.COM   
27423  Internet New Net Promo            1.0        WWW.HOPENERGY.COM   
27425  Internet New Net Promo            1.0    FLORALREFLECTIONS.NET   
27427  Internet New Net Promo            NaN                      NaN   

              PB_ID                     ST_DOMAIN  ST_MERGE_FLAG  \
20866  1.216995e+09           paradigmcapital.com            1.0   
20869  1.483139e+09                           NaN            NaN   
20870  1.320414e+09                 thebbland.com            1.0   
20872           NaN                           NaN            NaN   
20873  2.018667e+09                           NaN            NaN   
...             ...                           ...            ...   
27413  2.116478e+09                           NaN            NaN   
27414  2.115768e+09                           NaN            NaN   
27423  1.280870e+09                 hopenergy.com            1.0   
27425  1.130029e+09  floralreflectionsflowers.com            1.0   
27427           NaN                           NaN            NaN   

       ST_DUNSNUMBER    v12_web_address  V12_MERGE_FLAG  v12_company_id  \
20866    780514829.0         CLKING.COM             1.0      89019324.0   
20869            NaN        EUROPLAN.RU             1.0      53418809.0   
20870     80585068.0                NaN             NaN             NaN   
20872            NaN      NBLIEBMAN.COM             1.0      82589111.0   
20873            NaN                NaN             NaN             NaN   
...              ...                ...             ...             ...   
27413            NaN                NaN             NaN             NaN   
27414            NaN                NaN             NaN             NaN   
27423     27091980.0  WWW.DDMENERGY.COM             1.0      71975859.0   
27425    128636821.0                NaN             NaN             NaN   
27427            NaN       INFINITI.COM             1.0      63404922.0   

                          Final_url     segment merge  
20866           paradigmcapital.com  Mid-Market   yes  
20869                   EUROPLAN.RU  Mid-Market   yes  
20870                 thebbland.com  Mid-Market   yes  
20872                 NBLIEBMAN.COM  Mid-Market   yes  
20873          CERMELESCATERING.COM  Mid-Market   yes  
...                             ...         ...   ...  
27413        WWW.WARRENBEER.COM/WS/  Mid-Market   yes  
27414           WWW.WEISMARKETS.COM  Mid-Market   yes  
27423                 hopenergy

In [7]:
str(base['Final_url'][1]) == 'nan'

True

In [8]:
x =[[], [], [], [], [], [], [], [], [], [], [], ['Learn moreGoogle AdsenseADVERTISEMENTADVERTISEMENTADVERTISEMENTCustomer ServiceContact UsOrder StatusGive Us']] 

In [9]:
list(filter(None, x)) 

[['Learn moreGoogle AdsenseADVERTISEMENTADVERTISEMENTADVERTISEMENTCustomer ServiceContact UsOrder StatusGive Us']]

In [1]:
import pandas as pd

In [1]:
import pandas as pd
import nltk
nltk.download('punkt')

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\v.behl.snappoc\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True